In [ ]:
# %% [markdown]
# # 🎤 Simulation d'Entretien avec Groq (Llama 3.3 70B)
# 
# **Objectif** : Tester la génération de questions et l'évaluation des réponses
# 
# **Pipeline** :
# 1. Charger un CV et une offre
# 2. Générer des questions personnalisées
# 3. Simuler des réponses
# 4. Évaluer les réponses
# 5. Générer feedback global

In [1]:
# %%
import sys
sys.path.append('..')

import json
import os
from pathlib import Path
from src.interview_simulator import InterviewSimulator
from src.skills_extractor import SkillsExtractor
from src.cv_parser import CVParser

In [3]:
# %%
# CONFIGURATION : Clé API Groq
from dotenv import load_dotenv
import os

# Charger les variables depuis .env
load_dotenv()

# Vérifier que la clé est chargée
if not os.getenv("GROQ_API_KEY"):
    raise ValueError(
        "❌ GROQ_API_KEY non trouvée !\n"
        "1. Créez un fichier .env à la racine\n"
        "2. Ajoutez : GROQ_API_KEY=votre_clé\n"
        "3. Obtenez une clé sur : https://console.groq.com/"
    )

print(f"✅ Clé API chargée : {os.getenv('GROQ_API_KEY')[:20]}...")

# %%
# 1. INITIALISATION
print("="*60)
print("🚀 INITIALISATION")
print("="*60)

try:
    simulator = InterviewSimulator()
    skills_extractor = SkillsExtractor()
    print("✅ Tous les modules chargés")
except ValueError as e:
    print(f"❌ {e}")
    print("\n💡 Obtenez une clé gratuite sur : https://console.groq.com/")
    raise

✅ Clé API chargée : gsk_dQ0Dt8JrxyLkGtT2...
🚀 INITIALISATION
✅ InterviewSimulator initialisé avec llama-3.3-70b-versatile
✅ Modèle spaCy chargé
✅ Base de compétences chargée
   • Techniques : 171
   • Soft skills : 39
✅ Tous les modules chargés


In [4]:
# %%
# 2. CHARGER UN CV ET UNE OFFRE
print("\n" + "="*60)
print("📂 CHARGEMENT DES DONNÉES")
print("="*60)

# Charger le CV
cv_path = Path("../data/RESUME Robert UNG.pdf")

if not cv_path.exists():
    print(f"⚠️  CV introuvable : {cv_path}")
    print("Utilisez votre propre CV pour tester")
    # Simuler des compétences
    cv_skills = ["Python", "Machine Learning", "TensorFlow", "Docker", "FastAPI", "Git"]
else:
    parser = CVParser()
    cv_text = parser.parse(str(cv_path))
    skills_result = skills_extractor.extract_from_cv(cv_text)
    cv_skills = skills_result['technical_skills']

print(f"\n✅ Compétences CV : {', '.join(cv_skills[:10])}")

# Charger une offre
with open('../data/jobs/jobs_dataset.json', 'r', encoding='utf-8') as f:
    dataset = json.load(f)
    job = dataset['jobs'][0]  # ML Engineer Junior

print(f"\n✅ Offre sélectionnée")
print(f"   Titre : {job['title']}")
print(f"   Entreprise : {job['company']}")
print(f"   Requirements : {', '.join(job['requirements'][:5])}...")


📂 CHARGEMENT DES DONNÉES

✅ Compétences CV : artificial intelligence, big data, c, data analysis, data science, deep learning, docker, excel, git, java

✅ Offre sélectionnée
   Titre : Junior ML Engineer
   Entreprise : AI Startup Paris
   Requirements : Python (numpy, pandas, scikit-learn), Machine Learning basics (supervised learning), Git et GitHub, Docker (notions de base), Anglais technique (lecture documentation)...


In [5]:
# %%
# 3. GÉNÉRATION DES QUESTIONS
print("\n" + "="*60)
print("❓ GÉNÉRATION DES QUESTIONS")
print("="*60)

try:
    questions = simulator.generate_questions(
        cv_skills=cv_skills,
        job_title=job['title'],
        job_description=job['description'],
        job_requirements=job['requirements'],
        num_questions=8
    )
    
    print(f"\n✅ {len(questions['rh_questions'])} questions RH générées")
    print(f"✅ {len(questions['technical_questions'])} questions techniques générées")
    
except Exception as e:
    print(f"\n❌ ERREUR lors de la génération des questions:")
    print(f"   {type(e).__name__}: {e}")
    print("\n💡 Causes possibles:")
    print("   • Rate limiting Groq (429)")
    print("   • Clé API invalide")
    print("   • Problème de connexion")
    raise

print(f"\n📋 QUESTIONS RH ({len(questions['rh_questions'])}):")
for q in questions['rh_questions']:
    print(f"\n{q['id']}. [{q['type']}]")
    print(f"   {q['question']}")

print(f"\n🔧 QUESTIONS TECHNIQUES ({len(questions['technical_questions'])}):")
for q in questions['technical_questions']:
    skill = q.get('skill', 'N/A')
    print(f"\n{q['id']}. [{q['type']} - {skill}]")
    print(f"   {q['question']}")


❓ GÉNÉRATION DES QUESTIONS
✅ 4 questions RH générées
✅ 4 questions techniques générées

✅ 4 questions RH générées
✅ 4 questions techniques générées

📋 QUESTIONS RH (4):

1. [motivation]
   Quels sont vos objectifs de carrière à court et à long terme ?

2. [adaptabilité]
   Pouvez-vous me parler d'un moment où vous avez dû apprendre quelque chose de nouveau ?

3. [soft_skills]
   Comment gérez-vous les feedbacks ou les critiques constructives ?

4. [attentes]
   Qu'est-ce que vous attendez de notre équipe et de notre entreprise ?

🔧 QUESTIONS TECHNIQUES (4):

5. [compétence_technique - Pandas]
   Comment utilisez-vous les bibliothèques Pandas et NumPy pour l'analyse de données ?

6. [concept - Machine Learning]
   Expliquez les principes de base du Machine Learning et comment vous les appliquez

7. [outil - Docker]
   Décrivez votre expérience avec Docker et comment vous déployez des applications

8. [compétence_technique - Scikit-learn]
   Comment utilisez-vous Scikit-learn pour la cl

In [6]:
# %%
# 4. SIMULATION DE RÉPONSES
print("\n" + "="*60)
print("💬 SIMULATION DE RÉPONSES")
print("="*60)

# Exemples de réponses (bonnes, moyennes, mauvaises)
simulated_answers = {
    1: """Je suis diplômé d'un Master en IA et j'ai développé plusieurs projets en Machine Learning pendant mes stages. 
    J'ai notamment travaillé sur un système de recommandation avec TensorFlow qui a amélioré l'engagement de 15%. 
    Je suis passionné par l'apprentissage automatique et cherche à approfondir mes compétences en production de modèles ML.""",
    
    2: """Je souhaite rejoindre votre entreprise car vous êtes leaders dans l'IA. J'ai suivi vos publications techniques 
    et je suis impressionné par vos innovations. Cette position me permettrait d'apprendre auprès d'experts tout en 
    contribuant à des projets impactants.""",
    
    5: """En Python, j'utilise pandas pour la manipulation de données. Pour nettoyer un dataset, je commence par 
    df.dropna() pour les valeurs manquantes, puis df.drop_duplicates(). J'utilise aussi df.describe() pour les statistiques 
    et df.groupby() pour les agrégations. Pour l'optimisation, je préfère les opérations vectorisées aux boucles.""",
    
    6: """Pour une API REST, je structurerais avec FastAPI : des endpoints clairs (/users, /posts), 
    validation Pydantic, gestion d'erreurs avec HTTPException, et documentation auto Swagger. 
    J'ajouterais aussi du rate limiting et de l'authentification JWT."""
}

print("✅ 4 réponses simulées prêtes pour évaluation")


💬 SIMULATION DE RÉPONSES
✅ 4 réponses simulées prêtes pour évaluation


In [7]:
# %%
# 5. ÉVALUATION DES RÉPONSES
print("\n" + "="*60)
print("📊 ÉVALUATION DES RÉPONSES")
print("="*60)

evaluations = []

# Évaluer les 4 premières questions
test_questions = questions['rh_questions'][:2] + questions['technical_questions'][:2]

for question in test_questions:
    q_id = question['id']
    
    if q_id not in simulated_answers:
        continue
    
    print(f"\n{'='*60}")
    print(f"❓ Question {q_id}: {question['question'][:80]}...")
    print(f"\n💬 Réponse candidat:")
    print(f"{simulated_answers[q_id][:150]}...")
    
    # Évaluer avec Groq
    evaluation = simulator.evaluate_answer(
        question=question['question'],
        answer=simulated_answers[q_id],
        question_type=question['type'],
        target_skill=question.get('skill')
    )
    
    evaluations.append(evaluation)
    
    print(f"\n📈 RÉSULTAT:")
    print(f"   Score: {evaluation['score']:.0f}/100")
    print(f"   Évaluation: {evaluation['evaluation'][:200]}...")
    
    print(f"\n✅ Points forts:")
    for point in evaluation['points_forts'][:3]:
        print(f"   • {point}")
    
    print(f"\n⚠️  À améliorer:")
    for point in evaluation['points_amelioration'][:3]:
        print(f"   • {point}")


📊 ÉVALUATION DES RÉPONSES

❓ Question 1: Quels sont vos objectifs de carrière à court et à long terme ?...

💬 Réponse candidat:
Je suis diplômé d'un Master en IA et j'ai développé plusieurs projets en Machine Learning pendant mes stages. 
    J'ai notamment travaillé sur un sys...
✅ Réponse évaluée : 70/100

📈 RÉSULTAT:
   Score: 70/100
   Évaluation: Réponse claire et structurée, mais manque de détails sur les objectifs de carrière à court et long terme. Le candidat démontre une bonne compréhension de ses compétences techniques et une passion pour...

✅ Points forts:
   • Exemples concrets mentionnés, notamment le système de recommandation avec TensorFlow
   • Bonne structure de réponse, facile à suivre
   • Démontre une passion pour l'apprentissage automatique et une volonté d'approfondir ses compétences

⚠️  À améliorer:
   • Manque de détails sur les objectifs de carrière à court et long terme, notamment en termes de postes ou de secteurs
   • Pourrait ajouter des métriques chiffr

In [8]:
# %%
# 6. FEEDBACK GLOBAL
print("\n" + "="*60)
print("🎯 FEEDBACK GLOBAL")
print("="*60)

global_feedback = simulator.generate_final_feedback(
    evaluations=evaluations,
    job_title=job['title']
)

print(f"\n📊 Score global: {global_feedback['global_score']:.1f}/100")
print(f"🎯 Décision: {global_feedback['decision']}")

print(f"\n📝 Synthèse:")
print(f"{global_feedback['synthese']}")

print(f"\n✅ Compétences validées:")
for comp in global_feedback.get('competences_validees', [])[:5]:
    print(f"   • {comp}")

print(f"\n📈 Axes de progression:")
for axe in global_feedback.get('axes_progression', [])[:5]:
    print(f"   • {axe}")

print(f"\n🚀 Prochaines étapes:")
for etape in global_feedback.get('prochaines_etapes', [])[:5]:
    print(f"   • {etape}")


🎯 FEEDBACK GLOBAL
✅ Feedback global généré

📊 Score global: 52.5/100
🎯 Décision: Prometteur

📝 Synthèse:
Le candidat a montré un intérêt certain pour l'entreprise et le domaine du Machine Learning, avec des points forts notables tels que l'autonomie dans l'utilisation des bibliothèques et la motivation pour apprendre. Cependant, il y a des aspects qui nécessitent une amélioration, notamment en termes de détails techniques et de profondeur dans les réponses. Le candidat a également démontré une bonne structure de réponse et une passion pour l'apprentissage automatique.

✅ Compétences validées:
   • Intérêt pour l'entreprise et son domaine d'expertise
   • Autonomie dans l'utilisation des bibliothèques
   • Motivation pour apprendre et contribuer

📈 Axes de progression:
   • Approfondir les aspects techniques des réponses
   • Développer les compétences non techniques telles que la communication et la gestion d'équipe
   • Fournir des exemples concrets et des métriques chiffrées pour me

In [ ]:
# %% [markdown]
# ## ✅ Résumé
# 
# - ✅ **Groq (Llama 3.3 70B)** : Génération de questions ultra-rapide
# - ✅ **Questions personnalisées** : Basées sur CV + offre
# - ✅ **Évaluation automatique** : Scoring 0-100 avec feedback
# - ✅ **Feedback global** : Synthèse + recommandations
# 
# **Prochaine étape** : Intégrer dans l'API et Streamlit